# feature selection & modelling 

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import featuretools as ft


<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importli

In [2]:
print(pd.__version__)
print(np.__version__)
print(ft.__version__)

1.1.5
1.19.2
0.23.1


In [72]:
df_target_merged = pd.read_csv('./dataset/target_merged.csv')

/Users/billy/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3050: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [73]:
df_user_feature = pd.read_csv('./dataset/user_feature_total_remove_highcorr.csv')
df_merchant_feature = pd.read_csv('./dataset/merchant_feature_total_remove_highcorr.csv')
df_coupon_feature = pd.read_csv('./dataset/coupon_feature_total_remove_highcorr.csv')

In [74]:
df_user_feature = df_user_feature.add_prefix('user_')
df_merchant_feature = df_merchant_feature.add_prefix('merchant_')
df_coupon_feature = df_coupon_feature.add_prefix('coupon_')

## feature seletion 
To reduce modelling time and avoid the curse of dimensionality, feature selection is necessary for our project.

Traditional Methods of feature selection:

1. remove features with high missing rate 
2. remove features with singal value 
3. remove high correlated features 
4. univariate statistical test based 
5. model based

In [94]:
print('# user feature {}'.format(df_user_feature.shape[1]-2))
print('# merchant feature {}'.format(df_merchant_feature.shape[1]-2))
print('# coupon feature {}'.format(df_coupon_feature.shape[1]-2))

# user feature 73
# merchant feature 60
# coupon feature 39


In [95]:
df_target_merged[df_target_merged.data == 'Train']

,User_id,Merchant_id,Coupon_id,Date_received,Date,datediff,target,Month_received,data
0,1439408,4663,11002.0,2016-05-28,NaN,NaN,0.0,2016-05-01,Train
1,1439408,2632,8591.0,2016-02-17,NaN,NaN,0.0,2016-02-01,Train
2,1439408,2632,1078.0,2016-03-19,NaN,NaN,0.0,2016-03-01,Train
3,1439408,2632,8591.0,2016-06-13,NaN,NaN,0.0,2016-06-01,Train
4,1439408,2632,8591.0,2016-05-16,2016-06-13,28.0,-1.0,2016-05-01,Train
...,...,...,...,...,...,...,...,...,...
1053277,212662,3021,3739.0,2016-05-04,2016-05-08,4.0,1.0,2016-05-01,Train
1053278,212662,2934,5686.0,2016-03-21,2016-03-22,1.0,1.0,2016-03-01,Train
1053279,212662,3021,3739.0,2016-05-08,2016-06-02,25.0,-1.0,2016-05-01,Train
1053280,752472,7113,1633.0,2016-06-13,NaN,NaN,0.0,2016-06-01,Train


In [96]:
def cast_categorical_variable(df):
    cate_feat = list(
    filter(lambda x: 'MODE' in x , 
           df.columns)
    )
    df[cate_feat] = df[cate_feat].astype(object)
    return df

In [97]:
df_user_feature = cast_categorical_variable(df_user_feature)
df_merchant_feature = cast_categorical_variable(df_merchant_feature)
df_coupon_feature = cast_categorical_variable(df_coupon_feature)

In [98]:
df_train_target = df_target_merged[(df_target_merged['data'] == 'Train') & 
                                  (df_target_merged['target'] >= 0)][['User_id', 'Merchant_id',
                                     'Coupon_id', 'datediff','target','Month_received']]

In [140]:
df_train_target

,User_id,Merchant_id,Coupon_id,datediff,target,Month_received
0,1439408,4663,11002.0,NaN,0.0,2016-05-01
1,1439408,2632,8591.0,NaN,0.0,2016-02-01
2,1439408,2632,1078.0,NaN,0.0,2016-03-01
3,1439408,2632,8591.0,NaN,0.0,2016-06-01
5,1832624,3381,7610.0,NaN,0.0,2016-04-01
...,...,...,...,...,...,...
1053276,212662,3532,5267.0,NaN,0.0,2016-03-01
1053277,212662,3021,3739.0,4.0,1.0,2016-05-01
1053278,212662,2934,5686.0,1.0,1.0,2016-03-01
1053280,752472,7113,1633.0,NaN,0.0,2016-06-01


### statistical based feature selection
Now, we have about 180 variables with some null values and categorical variables. In this part, we will use WoE(Weight of Evidence) transformation to save us from encoding and imputing, which is widely used in research credit risk world.

blog: https://multithreaded.stitchfix.com/blog/2015/08/13/weight-of-evidence/

Steps:
1. to split (a continuous) variable into few categories or to group (a discrete) variable into few categories (and in both cases you assume that all observations in one category have "same" effect on dependent variable)
2. to calculate WoE value for each category (then the original x values are replaced by the WoE values)

Prons: (from the blog)
1. Seamlessly compare the strength of continuous and categorical variables without creating dummy variables.
2. Seamlessly handle missing values without imputation.
3. Assess the predictive power of missing values.

In this part, we use Python Package toad for WoE transformation and IV calculating.

Doc: https://toad.readthedocs.io/en/stable/

github: https://github.com/amphibian-dev/toad

In [99]:
import toad

from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

In [100]:
def woeTransform(df):
    ### binning data use chi2 binning algorithm, the minimum threshold in each bin is 2.5%
    ### return a combiner 
    c = toad.transform.Combiner() 
    df_binned = c.fit_transform(df, y = 'target', method = 'chi', min_samples = 0.025, n_bins=10) 
    woe_trans = toad.transform.WOETransformer()
    df_woe = woe_trans.fit_transform(df_binned,df_binned['target'],exclude=['target'])
    return c,woe_trans, df_woe

In [101]:
df_user = df_train_target.merge(right=df_user_feature, how='left', 
                                left_on= ['User_id','Month_received'],
                               right_on=['user_User_id','user_time'],
                                )
df_merchant_user = df_user.merge(right=df_merchant_feature, how='left',
                                left_on=['Merchant_id','Month_received'],
                               right_on=['merchant_Merchant_id','merchant_time'],
                                )
df_total = df_merchant_user.merge(right=df_coupon_feature, how='left',
                                left_on=['Coupon_id','Month_received'],
                               right_on=['coupon_Coupon_id','coupon_time'],
                                )

In [102]:
del df_user
del df_merchant_user

In [103]:
df_total

,User_id,Merchant_id,Coupon_id,datediff,target,Month_received,user_User_id,user_time,"user_AVG_TIME_BETWEEN(offline.Date, unit=days)","user_AVG_TIME_BETWEEN(offline.Date_received, unit=days)",...,coupon_NUM_UNIQUE(offline.WEEKDAY(Date)),coupon_NUM_UNIQUE(offline.WEEKDAY(Date_received)),coupon_NUM_UNIQUE(offline.WEEKDAY(Date_received) WHERE use_coupon = 1),coupon_SUM(offline.Distance WHERE use_coupon = 1),coupon_SUM(offline.Distance WHERE discount_type = 0.0),coupon_SUM(offline.datediff WHERE discount_type = 0.0),coupon_SUM(offline.promotion_amonut WHERE use_coupon = 1),coupon_SUM(offline.promotion_condition WHERE use_coupon = 1),coupon_SUM(offline.purchase WHERE discount_type = 1.0),coupon_SUM(offline.purchase WHERE discount_type = 0.0)
0,1439408,4663,11002.0,NaN,0.0,2016-05-01,1439408.0,2016-05-01,NaN,31.0,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1439408,2632,8591.0,NaN,0.0,2016-02-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1439408,2632,1078.0,NaN,0.0,2016-03-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1439408,2632,8591.0,NaN,0.0,2016-06-01,1439408.0,2016-06-01,NaN,70.0,...,2.0,7.0,3.0,3.0,0.0,0.0,3.0,60.0,3.0,0.0
4,1832624,3381,7610.0,NaN,0.0,2016-04-01,1832624.0,2016-04-01,NaN,NaN,...,4.0,6.0,4.0,2.0,0.0,0.0,160.0,1600.0,8.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042290,212662,3532,5267.0,NaN,0.0,2016-03-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1042291,212662,3021,3739.0,4.0,1.0,2016-05-01,212662.0,2016-05-01,5.5,0.5,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1042292,212662,2934,5686.0,1.0,1.0,2016-03-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1042293,752472,7113,1633.0,NaN,0.0,2016-06-01,752472.0,2016-06-01,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
df_train_target.shape

(1042295, 6)

In [105]:
features_list = list(df_user_feature.columns)[2:]+list(df_merchant_feature.columns)[2:]+list(df_coupon_feature.columns)[2:]

In [106]:
df_woe = df_total[features_list + ['target']]

In [108]:
#Conduct preliminary feature selection according to missing percentage, IV and correlation (with other features)
train_selected, dropped = toad.selection.select(df_woe,target = 'target',
                                                empty = 0.95, iv = 0.01, corr = 0.8,
                                                return_drop=True)
print(dropped)
print(train_selected.shape)

{'empty': array(['user_MEAN(online.uniform_discount_rate)',
       'user_MODE(online.Discount_rate WHERE Action = 2)',
       'user_NUM_UNIQUE(online.Merchant_id WHERE Action = 2)',
       'user_NUM_UNIQUE(online.WEEKDAY(Date_received) WHERE Action = 2)'],
      dtype='<U63'), 'iv': array(['user_MEAN(online.promotion_amonut)',
       'user_MEAN(online.promotion_condition)',
       'user_MODE(online.Discount_rate)',
       'user_MEAN(online.use_coupon WHERE Action = 1)',
       'user_MODE(online.WEEKDAY(Date) WHERE Action = 1)',
       'user_MODE(online.WEEKDAY(Date_received))',
       'user_NUM_UNIQUE(online.Coupon_id WHERE Action = 1)',
       'user_NUM_UNIQUE(online.Merchant_id WHERE Action = 1)',
       'user_NUM_UNIQUE(online.WEEKDAY(Date) WHERE Action = 1)',
       'user_NUM_UNIQUE(online.WEEKDAY(Date_received) WHERE Action = 1)',
       'merchant_MIN(merchant_offline.Distance WHERE discount_type = 1.0)'],
      dtype=object), 'corr': array(['merchant_SUM(merchant_offline.Distance

In [158]:
combiner,woe_transformer,transformed = woeTransform(train_selected)

In [ ]:
transformed.to_csv('./dataset/woe_feature_train.csv',index=False)

### MODEL based selection

Use Transformed feature for LogisticRegression, Random Forest, XGBoost

In [159]:
X = transformed.drop('target',axis=1)
y = transformed['target']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 1024,test_size=0.3)

In [160]:
print(X_train.shape)
print(X_test.shape)
print('Positive rate in train set:{:.2%}'.format(y_train.mean()))
print('Positive rate in validation set:{:.2%}'.format(y_test.mean()))

(729606, 116)
(312689, 116)
Positive rate in train set:6.19%
Positive rate in validation set:6.15%


#### LogisticRegression

In [119]:
selector = SelectFromModel(estimator=
                           LogisticRegression(solver='lbfgs',class_weight='balanced', max_iter=5000)
                          ,threshold='mean').fit(
    StandardScaler().fit_transform(X_train), y_train)

In [120]:
lr_features = set(X_train.columns[selector.get_support()])

In [121]:
len(lr_features)

39

#### Random Forest

In [122]:
rf = RandomForestClassifier(n_estimators=200, verbose=1, n_jobs=-1)

In [123]:
rf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  1.6min finished


RandomForestClassifier(n_estimators=200, n_jobs=-1, verbose=1)

In [124]:
rf_features = set(X_train.columns[rf.feature_importances_ > np.mean(rf.feature_importances_)])

In [125]:
len(rf_features)

50

#### XGBoost

In [126]:
xgb = XGBClassifier(n_estimators=200, max_depth=3,n_jobs=-1)

In [127]:
xgb.fit(X_train, y_train)

/Users/billy/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:56:11] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [128]:
xgb_features = set(X_train.columns[xgb.feature_importances_ >  np.mean(xgb.feature_importances_)])

### merge the results

In [81]:
result_features = list(lr_features.union(rf_features.union(xgb_features)))

In [191]:
len(result_features)

81

In [166]:
df_target_total = df_target_merged[['User_id', 'Merchant_id',
                                     'Coupon_id', 'datediff','target','Month_received','data']]

In [167]:
df_user = df_target_total.merge(right=df_user_feature, how='left', 
                                left_on= ['User_id','Month_received'],
                               right_on=['user_User_id','user_time'],
                                )
df_merchant_user = df_user.merge(right=df_merchant_feature, how='left',
                                left_on=['Merchant_id','Month_received'],
                               right_on=['merchant_Merchant_id','merchant_time'],
                                )
df_total = df_merchant_user.merge(right=df_coupon_feature, how='left',
                                left_on=['Coupon_id','Month_received'],
                               right_on=['coupon_Coupon_id','coupon_time'],
                                )

In [144]:
df_feature_selected = df_total[['User_id', 'Merchant_id','Coupon_id', 
                                'datediff','target','Month_received','data'] 
                               + result_features]

In [92]:
df_feature_selected.to_csv('feature_selected_train&test.csv',index=False)

In [168]:
df_total_woe = woe_transformer.transform(combiner.transform(df_total[result_features]))

In [174]:
df_total_woe = df_total_woe.round(4)

In [175]:
df_total_woe.to_csv('feature_selected_train&test_woe.csv',index=False)

In [176]:
cate_feat = list(
    filter(lambda x: 'MODE' in x , 
           df_feature_selected.columns)
    )

In [180]:
df_feature_selected[cate_feat] = df_total_woe[cate_feat]

In [187]:
df_feature_selected[cate_feat].to_csv('feature_selected_train&test_cate_woe.csv',index=False)

,user_NUM_UNIQUE(offline.Coupon_id WHERE purchase = 1),merchant_MEAN(merchant_offline.uniform_discount_rate),merchant_SUM(merchant_offline.purchase WHERE discount_type = 0.0),merchant_MODE(merchant_offline.WEEKDAY(Date)),merchant_MEAN(merchant_offline.purchase WHERE discount_type = 1.0),user_MODE(offline.Discount_rate WHERE give_coupon = 1),user_MODE(online.Action),"user_AVG_TIME_BETWEEN(offline.Date_received, unit=days)",user_MEAN(offline.promotion_amonut),coupon_SUM(offline.promotion_condition),...,user_MIN(offline.promotion_condition),user_MIN(offline.uniform_discount_rate),user_SUM(offline.purchase WHERE discount_type = 0.0),user_COUNT(online WHERE Action = 1),merchant_NUM_UNIQUE(merchant_offline.WEEKDAY(Date)),user_COUNT(offline),user_NUM_UNIQUE(offline.Merchant_id WHERE purchase = 1),user_MAX(offline.uniform_discount_rate),user_MAX(offline.Distance WHERE purchase = 1),user_NUM_UNIQUE(offline.WEEKDAY(Date))
0,0.5442,-1.3828,0.3918,0.3610,-0.3303,0.9630,-0.0544,0.4088,1.0096,0.5318,...,0.7949,0.7703,0.2266,0.2321,0.1337,0.3936,0.7633,0.8565,0.9511,0.2030
1,-0.2429,-0.0811,-0.2136,-0.1890,-0.0718,-0.1385,-0.0544,-0.0968,-0.1327,-0.2119,...,-0.1182,-0.1385,-0.2136,-0.2136,-0.1904,-0.2136,-0.2429,-0.1385,-0.2304,-0.2280
2,-0.2429,-0.0811,-0.2136,-0.1890,-0.0718,-0.1385,-0.0544,-0.0968,-0.1327,-0.2119,...,-0.1182,-0.1385,-0.2136,-0.2136,-0.1904,-0.2136,-0.2429,-0.1385,-0.2304,-0.2280
3,0.5442,0.7127,0.3918,1.0371,1.1825,-0.0188,-0.0544,0.4088,0.1464,0.9324,...,0.7949,0.1285,0.2266,0.2321,0.9070,0.3936,0.7633,0.8565,0.9511,0.2030
4,0.5442,0.7127,0.3918,1.0371,1.1825,0.9630,-0.0544,0.4088,1.0096,0.9324,...,0.7949,0.7703,0.2266,0.2321,0.9070,0.3936,0.7633,0.8565,0.9511,0.2030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166917,-0.2429,-0.0055,0.3918,0.2992,-0.2157,0.7868,-0.0544,0.9040,0.6595,0.5318,...,0.7949,0.8914,0.2266,0.2321,0.1337,0.2262,-0.2429,0.7416,-0.2304,-0.4746
1166918,1.3493,0.7127,0.3918,0.8315,0.2762,0.5441,-0.0544,0.9040,0.1464,-0.5684,...,0.4058,0.4953,0.2266,0.2321,0.8829,0.3936,0.7633,0.5760,-0.2304,0.2030
1166919,1.3493,0.7127,0.3918,-0.0065,-0.3303,0.5441,-0.0544,0.9040,0.1464,0.9324,...,0.4058,0.4953,0.2266,0.2321,0.1337,0.3936,0.7633,0.5760,-0.2304,0.2030
1166920,1.3493,0.0503,-0.3948,-0.0065,1.1825,0.7868,-0.0544,0.5941,0.6595,0.5318,...,-0.1182,0.4753,0.2266,0.2321,0.1337,0.6477,0.8454,0.6180,0.4422,0.9189
